In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

c:\Users\mirvo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer('sentence-transformers/LaBSE')

Downloading: 100%|██████████| 114/114 [00:00<00:00, 114kB/s]
Downloading: 100%|██████████| 2.36M/2.36M [00:00<00:00, 6.49MB/s]
Downloading: 100%|██████████| 1.62k/1.62k [00:00<00:00, 1.60MB/s]
Downloading: 100%|██████████| 122/122 [00:00<00:00, 122kB/s]
Downloading: 100%|██████████| 1.88G/1.88G [04:14<00:00, 7.40MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 52.9kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 112kB/s]
Downloading: 100%|██████████| 9.62M/9.62M [00:04<00:00, 2.31MB/s]
Downloading: 100%|██████████| 411/411 [00:00<00:00, 411kB/s]
Downloading: 100%|██████████| 5.22M/5.22M [00:03<00:00, 1.67MB/s]
Downloading: 100%|██████████| 461/461 [00:00<00:00, 461kB/s]


In [2]:
TAJIK_PATH = '..\\data\\tajik_titles_translated.csv'
PERSIAN_PATH = '..\\data\\persian_titles_translated.csv'

In [23]:
tajik = pd.read_csv(TAJIK_PATH, names=['id', 'title', 'some'])
persian = pd.read_csv(PERSIAN_PATH, names=['id', 'title', 'some'])

In [24]:
tajik['title'] = tajik['title'].astype(str) + tajik['some'].astype(str)
tajik = tajik.drop(columns=['some'])

persian['title'] = persian['title'].astype(str) + persian['some'].astype(str)
persian = persian.drop(columns=['some'])


In [26]:
TAJIK_EMBEDINGS_PATH = '..//data//tajik_titles_translated_embedings.csv'

tajik['embeding'] = tajik['title'].apply(lambda x: model.encode(x))
tajik.to_csv(TAJIK_EMBEDINGS_PATH)
del tajik

In [30]:
PERSIAN_EMBEDINGS_PATH = '..//data//persian_titles_translated_embedings.csv'

persian['embeding'] = persian['title'].apply(lambda x: model.encode(x))
persian.to_csv(PERSIAN_EMBEDINGS_PATH)
del persian

In [72]:
import ast

def embiding_parse(embeding_str: str):
    parsed_embeding_str = embeding_str\
        .replace('\n', ',')\
        .replace(' ', ',')\
        .replace(',,,,', ',')\
        .replace(',,,', ',')\
        .replace(',,', ',')\
        .replace('[,', '[')
    return ast.literal_eval(parsed_embeding_str)

In [73]:
tajik = pd.read_csv(TAJIK_EMBEDINGS_PATH)
tajik['embeding'] = tajik['embeding'].apply(lambda x: embiding_parse(x))

In [74]:
persian = pd.read_csv(PERSIAN_EMBEDINGS_PATH)
persian['embeding'] = persian['embeding'].apply(lambda x: embiding_parse(x))

In [86]:
def find_simulary(embeding):
    calc = persian['embeding'].apply(lambda x: np.dot(x, embeding))
    return [calc.max(), persian.loc[calc.idxmax()]['id']]

In [88]:
tajik.head(5)['embeding'].apply(find_simulary)

0    [0.7108551938220737, 10845]
1    [0.6146326864538375, 21702]
2    [0.4404012685231342, 20310]
3     [0.430628157451453, 19806]
4    [0.5018471236307425, 12116]
Name: embeding, dtype: object

In [89]:
tajik_prob = tajik['embeding'].apply(find_simulary)

In [104]:
tajik['res'] = tajik_prob
tajik['prob'] = tajik['res'].apply(lambda x: x[0])
tajik['persian_id'] = tajik['res'].apply(lambda x: x[1])

In [105]:
tajik = tajik.drop(columns=['res'])

,Unnamed: 0,id,title,embeding,prob,persian_id
0,0,10859,A sustainable attack on Baghdy has gone more t...,"[-0.0321400091, 0.0641805306, -0.0444514677, -...",0.710855,10845
1,1,1086,A group of group rapers was sentenced to the b...,"[0.0445780903, 0.0635005832, -0.0151647013, -0...",0.614633,21702
2,2,10860,Mavzuna took another gold medolnan,"[0.00691605126, 0.0484103076, 0.0226589106, 0....",0.440401,20310
3,3,10861,Heruin's discovery in a permanent stomachnan,"[-0.00294334465, -0.00197980227, 0.0366778001,...",0.430628,19806
4,4,10863,"Authorities of the officials for ""Leaders""nan","[-0.00164402, 0.01967564, 0.04868231, 0.005076...",0.501847,12116
...,...,...,...,...,...,...
9407,9407,10758,"""Afghanistan needs a Fedrottan"" nan","[0.01705284, 0.00621612, -0.0284916, 0.0435881...",0.625573,15264
9408,9408,10623,An acute security of security to hold games in...,"[-0.01556528, 0.03336932, 0.01535101, -0.01032...",0.459544,18593
9409,9409,10544,Organized people was attacked by the Chechen R...,"[0.00551287, 0.02863764, 0.00974519, -0.074530...",0.500836,22953
9410,9410,10697,Dozens of killed in the east of Tajikistan nan,"[0.0134077901, -0.0267296322, -0.0107943462, -...",0.748026,11054


In [110]:
tajik.drop(columns=['Unnamed: 0'])

,id,title,embeding,prob,persian_id
0,10859,A sustainable attack on Baghdy has gone more t...,"[-0.0321400091, 0.0641805306, -0.0444514677, -...",0.710855,10845
1,1086,A group of group rapers was sentenced to the b...,"[0.0445780903, 0.0635005832, -0.0151647013, -0...",0.614633,21702
2,10860,Mavzuna took another gold medolnan,"[0.00691605126, 0.0484103076, 0.0226589106, 0....",0.440401,20310
3,10861,Heruin's discovery in a permanent stomachnan,"[-0.00294334465, -0.00197980227, 0.0366778001,...",0.430628,19806
4,10863,"Authorities of the officials for ""Leaders""nan","[-0.00164402, 0.01967564, 0.04868231, 0.005076...",0.501847,12116
...,...,...,...,...,...
9407,10758,"""Afghanistan needs a Fedrottan"" nan","[0.01705284, 0.00621612, -0.0284916, 0.0435881...",0.625573,15264
9408,10623,An acute security of security to hold games in...,"[-0.01556528, 0.03336932, 0.01535101, -0.01032...",0.459544,18593
9409,10544,Organized people was attacked by the Chechen R...,"[0.00551287, 0.02863764, 0.00974519, -0.074530...",0.500836,22953
9410,10697,Dozens of killed in the east of Tajikistan nan,"[0.0134077901, -0.0267296322, -0.0107943462, -...",0.748026,11054


In [111]:
tajik.to_csv('..//data//tajik_titles_calced.csv')

In [109]:
tajik['prob'].describe()

count    9412.000000
mean        0.589102
std         0.108111
min         0.317549
25%         0.511486
50%         0.575388
75%         0.652383
max         0.992012
Name: prob, dtype: float64

In [116]:
PERSIAN_LINKS = '../data/beauty_links_persian_cleared.csv'
TAJIK_LINKS = '../data/beauty_links_tajik_cleared.csv'

In [117]:
persian_links = pd.read_csv(PERSIAN_LINKS)
tajik_links = pd.read_csv(TAJIK_LINKS)

In [120]:
persian_links = persian_links.drop(columns=['Unnamed: 0'])
tajik_links = tajik_links.drop(columns=['Unnamed: 0'])


In [121]:
tajik_links

,id,BeautyLink
0,0,https://www.bbc.com/tajik/news/2015/03/150331_...
1,1,https://www.bbc.com/tajik/news/2015/03/150331_...
2,2,https://www.bbc.com/tajik/news/2015/03/150330_...
3,3,https://www.bbc.com/tajik/news/2015/03/150329_...
4,4,https://www.bbc.com/tajik/news/2015/03/150329_...
...,...,...
12313,12330,https://www.bbc.com/tajik/news/2009/09/090922_...
12314,12331,https://www.bbc.com/tajik/news/2009/09/090922_...
12315,12332,https://www.bbc.com/tajik/news/2009/09/090922_...
12316,12333,https://www.bbc.com/tajik/news/2009/09/090922_...


In [ ]:
tajik = tajik.join(tajik_links.set_index('id'), on='id')
tajik = tajik.drop(columns=['Unnamed: 0'])

In [128]:
tajik

,id,title,embeding,prob,persian_id,BeautyLink
0,10859,A sustainable attack on Baghdy has gone more t...,"[-0.0321400091, 0.0641805306, -0.0444514677, -...",0.710855,10845,https://www.bbc.com/tajik/news/2010/08/100817_...
1,1086,A group of group rapers was sentenced to the b...,"[0.0445780903, 0.0635005832, -0.0151647013, -0...",0.614633,21702,https://www.bbc.com/tajik/news/2014/09/140907_...
2,10860,Mavzuna took another gold medolnan,"[0.00691605126, 0.0484103076, 0.0226589106, 0....",0.440401,20310,https://www.bbc.com/tajik/news/2010/08/100817_...
3,10861,Heruin's discovery in a permanent stomachnan,"[-0.00294334465, -0.00197980227, 0.0366778001,...",0.430628,19806,https://www.bbc.com/tajik/news/2010/08/100817_...
4,10863,"Authorities of the officials for ""Leaders""nan","[-0.00164402, 0.01967564, 0.04868231, 0.005076...",0.501847,12116,https://www.bbc.com/tajik/news/2010/08/100817_...
...,...,...,...,...,...,...
9407,10758,"""Afghanistan needs a Fedrottan"" nan","[0.01705284, 0.00621612, -0.0284916, 0.0435881...",0.625573,15264,https://www.bbc.com/tajik/news/2010/09/100907_...
9408,10623,An acute security of security to hold games in...,"[-0.01556528, 0.03336932, 0.01535101, -0.01032...",0.459544,18593,https://www.bbc.com/tajik/news/2010/10/101003_...
9409,10544,Organized people was attacked by the Chechen R...,"[0.00551287, 0.02863764, 0.00974519, -0.074530...",0.500836,22953,https://www.bbc.com/tajik/news/2010/10/101019_...
9410,10697,Dozens of killed in the east of Tajikistan nan,"[0.0134077901, -0.0267296322, -0.0107943462, -...",0.748026,11054,https://www.bbc.com/tajik/news/2010/09/100919_...


In [143]:
tajik = tajik.rename(columns={"BeautyLink": "link"})

In [134]:
persian_links

,id,BeautyLink
0,1,https://www.bbc.com/persian/iran/2015/03/15040...
1,2,https://www.bbc.com/persian/iran/2015/03/15040...
2,4,https://www.bbc.com/persian/world/2015/03/1503...
3,9,https://www.bbc.com/persian/iran/2015/03/15033...
4,11,https://www.bbc.com/persian/world/2015/03/1503...
...,...,...
37344,40579,https://www.bbc.com/persian/iran/2009/09/09090...
37345,40580,https://www.bbc.com/persian/iran/2009/09/09090...
37346,40581,https://www.bbc.com/persian/science/2009/09/09...
37347,40582,https://www.bbc.com/persian/sport/2009/09/0908...


In [ ]:
tajik = tajik.set_index('persian_id').join(persian_links.set_index('id'))

In [147]:
tajik = tajik.rename(columns={"BeautyLink": "persian_link"})

In [148]:
tajik

,id,title,embeding,prob,link,persian_link
11,11670,Attacking the voting trainer was lost in Baghd...,"[-0.0279040281, 0.0470479727, -0.028981857, -0...",0.545074,https://www.bbc.com/tajik/news/2010/03/100305_...,https://www.bbc.com/persian/world/2015/03/1503...
23,7397,Iran's talks and atumi reviewers remained unco...,"[0.0245353971, 0.0548132136, -0.0678865537, -0...",0.652886,https://www.bbc.com/tajik/institutional/2012/0...,https://www.bbc.com/persian/iran/2015/03/15033...
131,7474,"Iran's ""Katabi"" protest of the ""non-government...","[0.0187816825, 0.0137225213, 0.0216844473, -0....",0.586734,https://www.bbc.com/tajik/institutional/2012/0...,https://www.bbc.com/persian/iran/2015/03/15032...
131,7726,Eshart's response to Iran's request for Irania...,"[0.010930595, 0.0244756732, -0.0260604676, -0....",0.645492,https://www.bbc.com/tajik/institutional/2012/0...,https://www.bbc.com/persian/iran/2015/03/15032...
131,9044,Unprecedented attack of Irland Prime Minister ...,"[0.0535283238, -0.00944559928, -0.000848747266...",0.587920,https://www.bbc.com/tajik/institutional/2011/0...,https://www.bbc.com/persian/iran/2015/03/15032...
...,...,...,...,...,...,...
24229,557,Visit of organizations from designing changes ...,"[-0.0565002859, -0.0181033872, 0.0412582345, -...",0.472694,https://www.bbc.com/tajik/news/2014/12/141201_...,https://www.bbc.com/persian/world/2013/03/1303...
24229,6525,Request NGOs from the European Unionnan,"[-0.00445696, -0.02476437, 0.03187405, 0.01263...",0.489739,https://www.bbc.com/tajik/institutional/2012/0...,https://www.bbc.com/persian/world/2013/03/1303...
24229,10714,Realizing Human Rights Organizations from Kyrg...,"[0.0295441896, -0.0217680633, 0.0266988128, -0...",0.617925,https://www.bbc.com/tajik/news/2010/09/100916_...,https://www.bbc.com/persian/world/2013/03/1303...
24230,3478,Myprus buy the poor section of Niyanan,"[-0.0636667237, 0.0955320448, 0.0344631448, -0...",0.485521,https://www.bbc.com/tajik/news/2013/09/130903_...,https://www.bbc.com/persian/iran/2013/03/13032...


In [149]:
tajik.index.name = 'persian_id'
tajik = tajik.drop(columns=['embeding'])
tajik.to_csv('../data/matched-links.csv')

In [ ]:
# https://www.bbc.com/persian/sport/2013/09/130921_l51_wrestling_greco_roman_day2#TWEET897522
# https://www.bbc.com/tajik/institutional/2012/08/120806_rm_olympic_soryan_gold_wrestling
# 0.76